In [1]:
#!pip install python-speech-features

#### 模型预测与评估

In [2]:
# 语音识别预测
import automatic_speech_recognition as asr

# file = 'data/cv-valid-test/sample-000000.mp3'  # sample rate 16 kHz, and 16 bit depth
file = 'sample-en.wav'
# 加载语音数据
sample = asr.utils.read_audio(file)

# 加载模型
pipeline = asr.load('deepspeech2', lang='en')

# 打印模型结构
pipeline.model.summary()     # TensorFlow model

# 执行预测结果
sentences = pipeline.predict([sample])

# 打印预测结果
print(sentences)

Model: "DeepSpeech2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X (InputLayer)               [(None, None, 160)]       0         
_________________________________________________________________
lambda (Lambda)              (None, None, 160, 1)      0         
_________________________________________________________________
conv_1 (Conv2D)              (None, None, 80, 32)      14432     
_________________________________________________________________
conv_1_bn (BatchNormalizatio (None, None, 80, 32)      128       
_________________________________________________________________
conv_1_relu (ReLU)           (None, None, 80, 32)      0         
_________________________________________________________________
conv_2 (Conv2D)              (None, None, 40, 32)      236544    
_________________________________________________________________
conv_2_bn (BatchNormalizatio (None, None, 40, 32)      

In [1]:
# 语音识别预测
file = 'sample-en.wav'
sample = asr.utils.read_audio(file)
sentences = pipeline.predict([sample])
print(sentences)

NameError: name 'asr' is not defined

In [0]:
# 语音识别评估
test_dataset = asr.dataset.Audio.from_csv('data/cv-other-test.csv', batch_size=32)  # 载入语音数据集
wer, cer = asr.evaluate.calculate_error_rates(pipeline, test_dataset)  # 运行评估程序
print(f'WER: {wer}   CER: {cer}')  # 打印分数 WER 错字率， CER 错句率

/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


...

/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std
/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


WER: 0.40353817650709384   CER: 0.23137306327436663


#### 模型训练

In [3]:
import numpy as np
import tensorflow as tf
import automatic_speech_recognition as asr

dataset = asr.dataset.Audio.from_csv('data/cv-other-train.csv', batch_size=32)   # 载入训练数据集
alphabet = asr.text.Alphabet(lang='en')   # 选择语言模式
features_extractor = asr.features.FilterBanks(   # 设定特征结构
    features_num=160,
    winlen=0.02,
    winstep=0.01,
    winfunc=np.hanning
)
model = asr.model.get_deepspeech2(   # 设定模型参数，输入形状，输出形状
    input_dim=160,
    output_dim=29,
    rnn_units=800,
    is_mixed_precision=False
)
optimizer = tf.optimizers.Adam(   # 设定学习率
    lr=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-8
)
decoder = asr.decoder.GreedyDecoder()   # 设定解码器
pipeline = asr.pipeline.CTCPipeline(   # 定义模型
    alphabet, features_extractor, model, optimizer, decoder
)

pipeline.fit(dataset, epochs=20)   # 模型训练
pipeline.save('checkpoint')   # 模型保存

Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.
Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.
Epoch 1/20
 4535/4536 [===========================>..] - ETA: 2:20 - loss: 208.6205

/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


4536/4536 [==============================] - 35848s 8s/step - loss: 236.0765
Epoch 2/20
 4536/4536 [===========================>..] - ETA: 24:14 - loss: 208.2832

/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


4536/4536 [==============================] - 38498s 8s/step - loss: 181.6552
Epoch 3/20
 4536/4536 [===========================>..] - ETA: 5:20 - loss: 164.6186

...

4536/4536 [==============================] - 35649s 8s/step - loss: 3.5484
Epoch 19/20
 4536/4536 [===========================>..] - ETA: 32:08 - loss: 3.6898

/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


4536/4536 [==============================] - 35715s 8s/step - loss: 3.7845
Epoch 20/20
 4536/4536 [===========================>..] - ETA: 39:50 - loss: 3.4115

/content/Automatic-Speech-Recognition/automatic_speech_recognition/features/feature_extractor.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (features - mean) / std


4536/4536 [==============================] - 35577s 8s/step - loss: 3.6548
